In [8]:
import numpy as np
import pandas as pd
import torch
import os
import pickle
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, DataCollatorWithPadding
from transformers import BertForSequenceClassification, AdamW
import pytorch_lightning as pl
from transformers import BertTokenizer
from tqdm import tqdm
tqdm.pandas()

In [ ]:
#TODO: add proper requirements.txt

# Bert training (GPU necessary!)

In [ ]:
class LyricsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {'input_ids': self.encodings.iloc[idx] if isinstance(self.encodings.iloc[idx], torch.Tensor) else torch.tensor(self.encodings.iloc[idx])}
        item['labels'] = self.labels.iloc[idx] if isinstance(self.labels.iloc[idx], torch.Tensor) else torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.encodings)

class LyricsClassifier(pl.LightningModule):
    def __init__(self, model_name='bert-base-uncased', num_labels=5):
        super().__init__()
        self.save_hyperparameters()
        self.bert = BertForSequenceClassification.from_pretrained(self.hparams.model_name,
                                                                  num_labels=self.hparams.num_labels)

    def forward(self, input_ids, labels):
        return self.bert(input_ids, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self.forward(batch['input_ids'], batch['labels'])
        loss = outputs.loss
        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=1e-5)

### TODO: Callbacks for logging metrics and saving checkpoints


def load_data():
    with open("/content/drive/MyDrive/NLP/small_sample_tokenized/tokenized_lyrics.pickle", 'rb') as f:
        encodings = pickle.load(f)
    with open("/content/drive/MyDrive/NLP/small_sample_tokenized/labels_le.pickle", 'rb') as f:
        labels = pickle.load(f)

    return encodings, labels


def main():
    encodings, labels = load_data()

    #prepare tokenizer and data collator
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    #prepare datasets
    train_encodings, val_encodings, train_labels, val_labels = train_test_split(encodings, labels, test_size=0.1, random_state=42)
    train_dataset = LyricsDataset(train_encodings, train_labels)
    val_dataset = LyricsDataset(val_encodings, val_labels)

    model = LyricsClassifier()

    # data loaders
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=data_collator)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=data_collator)

    #Prepare trainer
    trainer = pl.Trainer(precision=16, limit_train_batches=0.5,max_epochs=3)

    # Training
    trainer.fit(model, train_loader, val_loader)
    
    return model

model = main()


In [ ]:
test_encodings, test_labels = load_test_data()  # assuming you have a function to load your test data
test_dataset = LyricsDataset(test_encodings, test_labels)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=data_collator)


accuracy, conf_matrix = evaluate(model, test_loader)
print("Accuracy: ", accuracy)
print("Confusion Matrix: ", conf_matrix)

# checking out performance with Checkpoint file

In [ ]:
#!pip install tensorboard

In [ ]:
# TODO: create test_encodings, test_labels for checkpoint testing

In [3]:
from transformers import pipeline
from sklearn.metrics import classification_report
import numpy as np

def evaluate_model(model, test_dataset):
    # Prepare data loader
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=data_collator)

    # Move model to evaluation mode
    model.eval()

    # Prepare prediction and label lists
    predictions = []
    actuals = []

    # Iterate through test data
    for batch in test_loader:
        # Move batch to device
        batch = {k: v.to(model.device) for k, v in batch.items()}
        # Forward pass
        with torch.no_grad():
            outputs = model(**batch)
        
        # Get the predicted labels
        _, preds = torch.max(outputs.logits, dim=1)

        # Move preds to CPU
        preds = preds.detach().cpu().numpy()
        predictions.extend(preds)

        # Extract labels and add to list
        labels = batch['labels'].detach().cpu().numpy()
        actuals.extend(labels)

    # Generate classification report
    report = classification_report(actuals, predictions, output_dict=True)

    return report

def load_and_evaluate_model(checkpoint_path, test_dataset):
    model = LyricsClassifier.load_from_checkpoint(checkpoint_path)

    report = evaluate_model(model, test_dataset)

    for label, metrics in report.items():
        print(f'\nClass: {label}')
        for metric, value in metrics.items():
            print(f'{metric}: {value}')



In [ ]:
# Usage example
test_dataset = LyricsDataset(test_encodings, test_labels)
load_and_evaluate_model('/content/lightning_logs/version_2/checkpoints/epoch=1-step=2812.ckpt', test_dataset)
